Some problems with the implementation proposed by Simeon Bamford:

-  the use of Euclidian distance -- we need to use Manhattan distance for my sake
-  Gaussian process for extracting $nProbConnect$ and others 
-  Source and target layers means either logical division (same "layers" on one core) or physical one with each layer on a different core
    
    
## Simulation Algorithm 1 Model Summary:

There are two layers of the size, the input and target layers; each is a square grid
of neurons with periodic boundaries, and the ideal location of each neuron in the input
layer is the location with the same coordinates in the target area. Each target-layer neuron
has the same number of potential synapses; these are dendritic locations in which actual
synapses may form; synapses can be with a pre-synaptic neuron from either the input or
target layer, including the post-synaptic neuron itself.

Initial conditions: all potential synapses start formed, with conductance $g_{max}$.

Input: neurons are independent Poisson processes. A stimulus location $s$ is randomly 
chosen and firing rates are set to 

$f_{base} + f_{peak}\times e^{-d / 2\sigma^{2}_{stim}}$
 
, where d is the distance from $s$. With a period $t_{sim}$, $s$ moves and the process repeats.

Neuron dynamics (target-layer): the membrane voltage $V_{mem}$ is described by:

$\tau_{mem}\frac{\delta V_{mem}}{\delta t} = V_{rest} - V_{mem} + g_{ex}(t)(E_{ex}-V_{mem})$


$E_{ex}$ = excitatory reversal potential; $V_{rest}$ = resting potential; $\tau_{mem}$ = membrane time constant. Upon reaching a threshold $V_{thr}$, a spike occurs and $V_{mem}$ is reset to $V_{rest}$. A presynaptic
spike at time 0 causes a synaptic conductance at time $t \geq O$ of $g_{ex}(t) = ge^\frac{-t}{\tau_{ex}}(\tau_{ex}$= synaptic time constant); this is cumulative for all pre-synaptic spikes.

**STDP**: a pre-synaptic spike at time $t_{pre}$ and post-synaptic spike at $t_{post}$ modify the synaptic
conductance by $g —> g + g_{max}F(\Delta t)$,where $\Delta t = t_{pre}—t_{post}$ and $F(\Delta t) = A_{+}exp (\Delta t/T_{+})$ if
$\Delta t$ < 0, otherwise $F(\Delta t) = -A_{-}exp (-\Delta t/T_{-})$, where $A_{+/-}$ are magnitudes and 
$\tau_{+/-}$ are time constants for potentiation and depression respectively. This is cumulative for all pre and
post-synaptic spike pairs; g is bounded in (0, $g_{max}$).

**Synaptic rewiring**: At a fixed rate $f_{rew}$, a potential synapse is randomly chosen. If it is an
actual synapse, the elimination rule is used, otherwise the formation rule is used.

**Formation rule**: A possible pre-synaptic neuron is randomly selected from either the input
or target layer and synapse formation occurs if:

$r < p_{form}e^{-\frac{\delta^{2}}{2\sigma^2_{form}}}$

r = uniform random number in (0, 1); $p_{form}$ = peak formation probability;$\delta$ = distance of 
possible pre-synaptic neuron from ideal location of post-synaptic neuron; $\sigma_{form}$  = variance 
of the connection field. $p_{form}$ and $\sigma^2_{form}$ formmay differ based on which layer the possible presynaptic
neuron is from.

**Elimination rule**: If the synapse's conductance is below $0.5g_{max}$ it is eliminated with probability
$p_{elim-dep}$, otherwise probability $p_{elim-pot}$ is used
   

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Layer representation

### Periodic Boundries 